In [ ]:
!pip install chordparser #necessary in order to read chords into roman numeral form

Managing dependencies

In [ ]:
import os
import chordparser
import pandas as pd
import re

Necessary imports

In [ ]:
def verse_string_key(path):
  first_chord_line = 7 #defining the first line where a chord can be found in the SALAMI text files
  key_line = [3]       #defining the line where the key of the piece is enumerated
  success = 0          #debug variable

  #Next, we loop through the directory argument, and attempt to find a pipe (|) character.
  #We then look for the string "verse" to verify that the line describes a verse of
  #the song; if it isn't found, we search for the first chorus line instead.
  for x, line in enumerate(open(path)):
    if (x >= first_chord_line):
      #print("x in chord_lines")
      first_pipe_index = line.lower().find("verse")
      if (first_pipe_index != -1):
        verse_string = line[first_pipe_index:]
        #print("break")
        success = 1
        break

    if (success == 0):
      for x, line in enumerate(open(path)):
        if (x >= first_chord_line):
            first_pipe_index = line.lower().find("chorus")
            if (first_pipe_index != -1):
              verse_string = line[first_pipe_index:]
              #print("break")
              success = 1
              break


  key_list = [] #define a list that will contain each character of the key of the piece
  for x, line in enumerate(open(path)):
    if (x in key_line):
      for y in line[::-1]:
        #print(y)

        #if a space character is encountered, break from the loop.
        if (y != " "):
          key_list = [y] + key_list
        else:
          break

  #for some reason I haven't yet identified, the \n escape sequence keeps appearing in the key_list,
  #so we remove it with this loop.
  for x in range(len(key_list)):
    if (key_list[x] == '\n'):
      key_list.pop(x)

  #now that we have cleaned up key_list, we can join the list into a string
  key = "".join(key_list)

  #print("\n" + verse_string)
  print(key)

  return verse_string, key #the line of the verse (or chorus) is returned, along with the key signature of the piece

Finding the first line of the SALAMI file containing the chords of the verse

In [ ]:
def find_pipes(verse_string):
  verse_string_copy = verse_string
  verse_string_list = list(verse_string)
  pipes = []
  for i in range(5):
    pipes.append(verse_string_copy.find("|"))
    verse_string_list[pipes[i]] = "&"
    verse_string_copy = "".join(verse_string_list)
  pipes.append(len(verse_string)-1)

  return pipes #a list containing the indices of the pipes is returned

Finding the indices of the pipes in the line containing the verse's chords

In [ ]:
def verse_string_to_verse_chords(verse_string, pipes):
  verse_chords = [] #should be a list of strings
  for i in range(4):
    verse_chords.append(verse_string[pipes[i]+2:pipes[i+1]-1])

  #print(verse_chords)

  for x in range(len(verse_chords)):
    for i in range(len(verse_chords[x])):
      try:
        if (verse_chords[x][i] == " "):
          verse_chords[x] = verse_chords[x][0:i]
      except:
        break

  #print(verse_chords)

  return verse_chords

Creating a list of the chords present in the verse_string

In [ ]:
def create_data(path):
  verse_string, key = verse_string_key(path)
  pipes = find_pipes(verse_string)
  verse_chords = verse_string_to_verse_chords(verse_string, pipes)
  #print(verse_chords)
  return verse_chords, key

All methods defined so far are combined for simplicity's sake

In [ ]:
def clean_data(verse_chords):
  for x in range(len(verse_chords)):
    verse_chords[x] = re.sub(":1", '', verse_chords[x])           #Removing lone bass notes
    verse_chords[x] = re.sub(r'\([^()]*\)', '', verse_chords[x])  #Removing unwanted symbols
    verse_chords[x] = re.sub(":", "", verse_chords[x])            #Removing colons
    verse_chords[x] = re.sub("\(", "", verse_chords[x])           #Removing opening parenthesis
    verse_chords[x] = re.sub("\)", "", verse_chords[x])           #Removing closing paranthesis

    #Finally, we convert all occurrences of 'hdim' into 'b5', which can be read by chordparser.
    if (re.findall("hdim", verse_chords[x])):
      verse_chords[x] = re.sub("hdim", "", verse_chords[x])
      verse_chords[x] += "b5"

Removing unwanted characters from data

In [ ]:
def chords_to_roman(cleaned_output, key):
  for i in range(len(cleaned_output)):
    chord = parser.create_chord(cleaned_output[i])
    cleaned_output[i] = parser.to_roman(chord, key)
  print(cleaned_output)
  return cleaned_output

Converting the output from the previous method to roman numeral chords

In [ ]:
df = pd.DataFrame([[1, 2, 3, 4]], columns=["Chord1",  "Chord2", "Chord3", "Chord4"])
print(df)

   Chord1  Chord2  Chord3  Chord4
0       1       2       3       4


Creating a dataframe (in order to later convert to csv)

In [ ]:
def write_data_csv(path):
  num_of_songs = 0
  num_of_4_4_songs = 0
  data = pd.DataFrame(columns=["Chord1",  "Chord2", "Chord3", "Chord4"])
  meter_line = [2] #the line of the salami_chords.txt files that contains song's metre


  for f in os.scandir(path): #iterate through main directory
    num_of_songs += 1
    file_path = [x.path for x in os.scandir(f.path)] #iterate through subdirectory
    try:
      file_path_string = file_path[0]
    except:
      continue
    print(file_path_string)

    try:
      for x, line in enumerate(open(file_path_string)): #iterate through lines
        if (x in meter_line):

          if (line[len(line)-4:len(line)-1] == "4/4"): #check if song is in 4/4
            num_of_4_4_songs += 1

            verse_chords, key = create_data(file_path_string)
            cleaned_output = clean_data(verse_chords)
            key = parser.create_key(key)
            roman_chords = chords_to_roman(cleaned_output, key)
            new_data = pd.DataFrame([roman_chords], columns=["Chord1",  "Chord2", "Chord3", "Chord4"])
            print(new_data)
            data = data.append(new_data)

    except:
      print("exception\n")
    #print(data)

    if (num_of_4_4_songs % 50 == 0): #keeping track of how close to completion we are
      print(num_of_4_4_songs)

  data.to_csv("data.csv.gz", index=False, compression="gzip")

Creating one method to rule them all! (Throwing it all together into one method)

In [ ]:
path = "../content/drive/MyDrive/McGill-Billboard"
parser = chordparser.Parser()
write_data_csv(path)

../content/drive/MyDrive/McGill-Billboard/0677/salami_chords.txt
D
[i7 roman chord, i7 roman chord, i7 roman chord, i7 roman chord]
  Chord1 Chord2 Chord3 Chord4
0     i7     i7     i7     i7
../content/drive/MyDrive/McGill-Billboard/0684/salami_chords.txt
F
[V roman chord, ♭VII roman chord, ♭VII roman chord, ♭VII roman chord]
  Chord1 Chord2 Chord3 Chord4
0      V   ♭VII   ♭VII   ♭VII
../content/drive/MyDrive/McGill-Billboard/0680/salami_chords.txt
D
[I roman chord, I roman chord, ii roman chord, ii°7 roman chord]
  Chord1 Chord2 Chord3 Chord4
0      I      I     ii   ii°7
../content/drive/MyDrive/McGill-Billboard/0663/salami_chords.txt
C
[iv9 roman chord, i roman chord, iv9 roman chord, ♭VII roman chord]
  Chord1 Chord2 Chord3 Chord4
0    iv9      i    iv9   ♭VII
../content/drive/MyDrive/McGill-Billboard/0664/salami_chords.txt
C
[I roman chord, I roman chord, IV roman chord, IV roman chord]
  Chord1 Chord2 Chord3 Chord4
0      I      I     IV     IV
../content/drive/MyDrive/McGill-Bi

/usr/local/lib/python3.6/dist-packages/chordparser/editors/chord_roman_converter.py:83: UserWarning: Warning: Power and sus chords are defaulted to major chords
  UserWarning


../content/drive/MyDrive/McGill-Billboard/0650/salami_chords.txt
C
[IM7 roman chord, IM7 roman chord, vi roman chord, vi roman chord]
  Chord1 Chord2 Chord3 Chord4
0    IM7    IM7     vi     vi
../content/drive/MyDrive/McGill-Billboard/0655/salami_chords.txt
Ab
[III roman chord, ♯v7 roman chord, VI roman chord, ♯iv7 roman chord]
  Chord1 Chord2 Chord3 Chord4
0    III    ♯v7     VI   ♯iv7
../content/drive/MyDrive/McGill-Billboard/0658/salami_chords.txt
../content/drive/MyDrive/McGill-Billboard/0649/salami_chords.txt
D
[I roman chord, vi roman chord, ii roman chord, I roman chord]
  Chord1 Chord2 Chord3 Chord4
0      I     vi     ii      I
../content/drive/MyDrive/McGill-Billboard/0656/salami_chords.txt
../content/drive/MyDrive/McGill-Billboard/0640/salami_chords.txt
Eb
[I7 roman chord, I7 roman chord, I7 roman chord, I7 roman chord]
  Chord1 Chord2 Chord3 Chord4
0     I7     I7     I7     I7
../content/drive/MyDrive/McGill-Billboard/0638/salami_chords.txt
G
[I roman chord, I7 roman chor

/usr/local/lib/python3.6/dist-packages/chordparser/editors/chord_roman_converter.py:83: UserWarning: Warning: Power and sus chords are defaulted to major chords
  UserWarning


../content/drive/MyDrive/McGill-Billboard/0636/salami_chords.txt
../content/drive/MyDrive/McGill-Billboard/0635/salami_chords.txt
Eb
[I roman chord, I roman chord, I roman chord, IV roman chord]
  Chord1 Chord2 Chord3 Chord4
0      I      I      I     IV
../content/drive/MyDrive/McGill-Billboard/0643/salami_chords.txt
Ab
[i roman chord, IV roman chord, V7 roman chord, V7 roman chord]
  Chord1 Chord2 Chord3 Chord4
0      i     IV     V7     V7
../content/drive/MyDrive/McGill-Billboard/0633/salami_chords.txt
B
[♭III roman chord, ♭III roman chord, ♭VII roman chord, ♭VII roman chord]
  Chord1 Chord2 Chord3 Chord4
0   ♭III   ♭III   ♭VII   ♭VII
../content/drive/MyDrive/McGill-Billboard/0637/salami_chords.txt
D
[IV roman chord, IV roman chord, IV roman chord, IV roman chord]
  Chord1 Chord2 Chord3 Chord4
0     IV     IV     IV     IV
../content/drive/MyDrive/McGill-Billboard/0645/salami_chords.txt
G
[I roman chord, III roman chord, IV roman chord, II roman chord]
  Chord1 Chord2 Chord3 Chord4

...

../content/drive/MyDrive/McGill-Billboard/0698/salami_chords.txt
D
exception

../content/drive/MyDrive/McGill-Billboard/0699/salami_chords.txt
E
[♭VII roman chord, ♭VII roman chord, IV roman chord, IV roman chord]
  Chord1 Chord2 Chord3 Chord4
0   ♭VII   ♭VII     IV     IV
../content/drive/MyDrive/McGill-Billboard/0690/salami_chords.txt
D
[I roman chord, I roman chord, I roman chord, I roman chord]
  Chord1 Chord2 Chord3 Chord4
0      I      I      I      I
../content/drive/MyDrive/McGill-Billboard/0692/salami_chords.txt
A
[i roman chord, i roman chord, i roman chord, i roman chord]
  Chord1 Chord2 Chord3 Chord4
0      i      i      i      i
../content/drive/MyDrive/McGill-Billboard/0688/salami_chords.txt
E
[I7 roman chord, I7 roman chord, I roman chord, I7 roman chord]
  Chord1 Chord2 Chord3 Chord4
0     I7     I7      I     I7
800
../content/drive/MyDrive/McGill-Billboard/0696/salami_chords.txt
Bb
[I roman chord, VI7 roman chord, II7 roman chord, II7 roman chord]
  Chord1 Chord2 Chor

/usr/local/lib/python3.6/dist-packages/chordparser/editors/chord_roman_converter.py:83: UserWarning: Warning: Power and sus chords are defaulted to major chords
  UserWarning


G
[I roman chord, I roman chord, vi9 roman chord, vi9 roman chord]
  Chord1 Chord2 Chord3 Chord4
0      I      I    vi9    vi9
../content/drive/MyDrive/McGill-Billboard/0687/salami_chords.txt
C
exception

../content/drive/MyDrive/McGill-Billboard/0675/salami_chords.txt


/usr/local/lib/python3.6/dist-packages/chordparser/editors/chord_roman_converter.py:83: UserWarning: Warning: Power and sus chords are defaulted to major chords
  UserWarning


E
[I roman chord, ii roman chord, ii roman chord, I roman chord]
  Chord1 Chord2 Chord3 Chord4
0      I     ii     ii      I
../content/drive/MyDrive/McGill-Billboard/0683/salami_chords.txt
Ab
[i9 roman chord, i roman chord, i roman chord, iv9 roman chord]
  Chord1 Chord2 Chord3 Chord4
0     i9      i      i    iv9
